<a href="https://colab.research.google.com/github/comojin1994/Deep_Learning_Study/blob/master/3step_lecture/Training_shallow_NN_with_GD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
tf.__version__

TensorFlow 2.x selected.


'2.1.0'

### Hyperparameter

In [0]:
EPOCHS = 1000

### Model

In [0]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.d1 = tf.keras.layers.Dense(128, input_dim=2, activation='sigmoid', kernel_initializer='he_normal')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax', kernel_initializer='he_normal')
            
    def call(self, x, training=None, mask=None):
        x = self.d1(x)
        x = self.d2(x)
        return x

### Define train step

In [0]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
  with tf.GradientTape() as tape:
    predictions = model(inputs)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_metric(labels, predictions)

### Make Dataset

In [0]:
np.random.seed(0)

pts = list()
labels = list()
center_pts = np.random.uniform(-8.0, 8.0, (10, 2))
for label, center_pt in enumerate(center_pts):
  for _ in range(100):
    pts.append(center_pt + np.random.randn(*center_pt.shape))
    labels.append(label)
pts = np.stack(pts, axis=0).astype(np.float32)
labels = np.stack(labels, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((pts, labels)).shuffle(1000).batch(32)

### Make Model

In [0]:
model = MyModel()

### Define loss function & Optimizer

In [7]:
!pip install keras-rectified-adam

  Created wheel for keras-rectified-adam: filename=keras_rectified_adam-0.17.0-cp36-none-any.whl size=14781 sha256=3af2700224e4a63472f1725114500402d1608adab7c3c56d2e985334bda8da0f
  Stored in directory: /root/.cache/pip/wheels/7b/01/27/3a934e1a5644f5b93c720422a6ef97034ea78a21ba71cfb549
Successfully built keras-rectified-adam


In [8]:
from keras_radam.training import RAdamOptimizer as RAdam

Using TensorFlow backend.


In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
# optimizer = tf.keras.optimizers.Adam()
optimizer = RAdam()

### Metric

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

### Make Train loop

In [11]:
for epoch in range(EPOCHS):
  for x, label in train_ds:
    train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)

  template = 'Epoch {}, Loss: {}, Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result() * 100))

Epoch 1, Loss: 2.77199387550354, Accuracy: 10.100000381469727
Epoch 2, Loss: 2.69461727142334, Accuracy: 12.800000190734863
Epoch 3, Loss: 2.6026899814605713, Accuracy: 14.933332443237305
Epoch 4, Loss: 2.5018186569213867, Accuracy: 16.82499885559082
Epoch 5, Loss: 2.397925615310669, Accuracy: 19.34000015258789
Epoch 6, Loss: 2.296220064163208, Accuracy: 22.666667938232422
Epoch 7, Loss: 2.200254201889038, Accuracy: 25.885713577270508
Epoch 8, Loss: 2.1119775772094727, Accuracy: 29.350000381469727
Epoch 9, Loss: 2.0304224491119385, Accuracy: 33.255558013916016
Epoch 10, Loss: 1.9564332962036133, Accuracy: 36.709999084472656
Epoch 11, Loss: 1.889149785041809, Accuracy: 39.45454788208008
Epoch 12, Loss: 1.8276664018630981, Accuracy: 42.0
Epoch 13, Loss: 1.7713427543640137, Accuracy: 44.238460540771484
Epoch 14, Loss: 1.7197810411453247, Accuracy: 46.235713958740234
Epoch 15, Loss: 1.671410322189331, Accuracy: 48.12000274658203
Epoch 16, Loss: 1.6269326210021973, Accuracy: 49.706249237060

### Save

In [0]:
np.savez_compressed('ch2_dataset.npz', inputs=pts, labels=labels)

W_h, b_h = model.d1.get_weights()
W_o, b_o = model.d2.get_weights()
W_h = np.transpose(W_h)
W_o = np.transpose(W_o)
np.savez_compressed('ch2_parameters.npz',
                    W_h = W_h,
                    b_h = b_h,
                    W_o = W_o,
                    b_o = b_o)